# 协程

In [5]:
import time
def fn(name):
    print("Hello")
    time.sleep(3)
    print(name)

name=input()
fn(name)

Hello
ha


上面这个程序很好地呈现了什么是“阻塞”。所谓“阻塞”，就是程序运行到如sleep()或input()这里，因为需要强制休眠或者等待输入而导致CPU停止了工作（通过暂停工作来完成任务）。同理，在requests.get()中，网络请求返回数据之前，程序也是处于阻塞状态的。其实一般情况下，程序在执行IO操作的时候同样也是处于阻塞状态的。

**协程**：当遇见了io操作的时候，可以选择性地切换到其他任务上。

在微观上是一个任务一个任务地进行切换，切换条件就是发生io操作；在宏观上我们能看到实际上是多个任务一起执行。这就是多任务异步操作。上面说的一切都是在单线程的条件下。

In [ ]:
import asyncio

async def func():           #添加async:变成了异步协程函数
    print("nihao!")

if __name__=='__main__':
    g=func()                #此时函数执行得到的是一个协程对象
    # print(g)
    asyncio.run(g)          #不知道为什么这里会报错，难道是因为前面的程序的原因吗？所以现在同名py文件下运行。

接下来演示如何使用asyncio进行异步协程加快执行速度。

In [ ]:
import asyncio
import time
async def fn():           #添加async:变成了异步协程函数
    print("nihao!")

async def func1():
    print("i am ")
    # time.sleep(2)         #同步休眠，休眠的时候不能执行其他任何任务
    await asyncio.sleep(2)  #异步的休眠，这个任务休眠的时候立刻运行其他的任务。
    print('func1')

async def func2():
    print("i am ")
    # time.sleep(2)
    await asyncio.sleep(2)
    print('func2')

async def func3():
    print("i am ")
    # time.sleep(2)
    await asyncio.sleep(2)
    print('func3')

if __name__=='__main__':
    # g=fn()#此时函数执行得到的是一个协程对象
    # # print(g)
    # asyncio.run(g)  #协程程序的运行需要asyncio的支持
    f1=func1()
    f2=func2()
    f3=func3()
    tasks=[f1,f2,f3]
    t1=time.time()
    asyncio.run(asyncio.wait(tasks))    #多个任务交给asyncio.wait，用asyncio.run执行
    t2=time.time()
    print("time:",t2-t1)               
     #使用time.sleep():time: 6.01953387260437:说明sleep还是没有避免，出现了同步操作的时候，异步操作就中断了。
     #使用await asyncio.sleep():time: 2.0174758434295654，此时效率得到了大幅提升。最终的等待时间被缩减为2秒。

这个程序虽然确实提高了执行速度，但是这不是最好的方案。下面给出一个python官方推荐的异步协程程序的写法。

In [ ]:
import asyncio
import time
async def fn():           #添加async:变成了异步协程函数
    print("nihao!")

async def func1():
    print("i am ")
    # time.sleep(2)         #同步休眠，休眠的时候不能执行其他任何任务
    await asyncio.sleep(2)  #异步的休眠，这个任务休眠的时候立刻运行其他的任务。
    print('func1')

async def func2():
    print("i am ")
    # time.sleep(2)
    await asyncio.sleep(2)
    print('func2')

async def func3():
    print("i am ")
    # time.sleep(2)
    await asyncio.sleep(2)
    print('func3')
async def main():
    # #python要求：1.await挂起操作必须放在这个定义的main里面。2.await必须放在协程对象前面。
    # #第一种写法，并不推荐：
    # f1=func1()
    # await f1...        #await挂起实际是调用一个异步协程对象，相当于就让f1运行起来了

    # tasks=[func1(),func2(),func3()]
    #为了应对python3.11即将废除这种直接把协程对象放到列表里边的问题，以后需要手动在tasks里面创建tasks对象。
    tasks=[
        asyncio.create_task(func1()),
        asyncio.create_task(func2()),
        asyncio.create_task(func3())
    ]
    
    await asyncio.wait(tasks)   
    #await是挂起操作，指提上日程开始执行；asyncio.wait指的是task里面的函数应该以异步协程的方式运行。
    #之所以推荐这种方法，是因为这一套模板能够很完美地适配一个爬虫。
    #我们可以把不同的url封装在函数里面，形成一个函数组，放进一个tasks列表然后直接套到这个模板里边。
    #效率极高。
    pass
if __name__=='__main__':
    t1=time.time()
    asyncio.run(main())
    t2=time.time()
    print('time',t2-t1)#time 2.010483503341675

In [10]:
import requests
from lxml import etree
url='https://www.biqudd.com/126_126944/53542058.html'
name='essays/'+url.rsplit("/",1)[1].replace(".html",".txt")
print(name)
# resp=requests.get(url)
# tree=etree.HTML(resp.text)
# contents=tree.xpath('/html/body/div[1]/div[2]/div/div[4]/text()')
# contstr=''
# for cont in contents[1:]:
#     cont=cont.strip('\xa0')
#     contstr+=cont
#     contstr+='\n'
# print(contstr)


essays/53542058.txt
